In [36]:
import keyring

SERVICE = 'OpenAI'
NAME = 'HomeworkAPIKey'
SERVICE2 = 'SerpAPI'
NAME2 = 'APIKey'

apikey = keyring.get_password(SERVICE, NAME)
apikey2 = keyring.get_password(SERVICE2, NAME2)

In [6]:
from langchain.llms import OpenAI

# Temperature = creativity
# 0 = play it safe
# 1 = super creative but higher risk of hallucination
llm = OpenAI(temperature=0.6, api_key=apikey)
name = llm('I want to open a restaurant for Italian food. Suggest a fancy name for this.')
print(name)



"La Dolce Vita"


In [14]:
from langchain.prompts import PromptTemplate

import warnings
from langchain_core._api.deprecation import LangChainDeprecationWarning

warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.'
)
prompt_template_name.format(cuisine='Mexican')

'I want to open a restaurant for Mexican food. Suggest a fancy name for this.'

In [16]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run('Polish')

'\n"Polish Delights" or "Polskie Piękności" (Polish Beauties)'

In [17]:
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = '''Suggest some menu items for {restaurant_name}. Return it as a comma separted list.'''
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [18]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
response = chain.run('Indian')
print(response)

# Note:  Don't get restaurant name!



1. Samosas
2. Chicken Tikka Masala
3. Lamb Vindaloo
4. Palak Paneer
5. Vegetable Biryani
6. Tandoori Shrimp
7. Garlic Naan
8. Mango Lassi
9. Chana Masala
10. Butter Chicken
11. Aloo Gobi
12. Dal Makhani
13. Chicken Korma
14. Vegetable Curry
15. Gulab Jamun (dessert)


In [19]:
# Increasing creativity level:
llm = OpenAI(temperature=0.7, api_key=apikey)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='restaurant_name')
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key='menu_items')

In [20]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)
chain({'cuisine': 'Arabic'})

{'cuisine': 'Arabic',
 'restaurant_name': '\n\n"Saffron Palace"',
 'menu_items': '\n\n1. Saffron Chicken Tikka Masala\n2. Vegetable Biryani\n3. Lamb Korma\n4. Tandoori Shrimp\n5. Palak Paneer\n6. Garlic Naan\n7. Mango Lassi\n8. Samosas\n9. Chicken Vindaloo\n10. Dal Makhani'}

In [24]:
from langchain.agents import AgentType, initialize_agent, load_tools

# Google for langchain agent load tools to get list in docs:
# Currently:  https://python.langchain.com/docs/integrations/tools/
tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run('When was Elon Musk born?  What is his age as of March 23, 2025?')



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information on Elon Musk
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman known for his key roles in Tesla, SpaceX, and Twitter (which he rebranded as X). Since 2025, he has been a senior advisor to United States president Donald Trump and the de facto head of the Department of Government Efficiency (DOGE). Musk is the wealthiest person in the world; as of March 2025, Forbes estimates his net worth to be $320 billion USD. 
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada, whose citizenship he had inherited through his mother. He graduated from the University of Pennsylvania in the U.S. before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to 

'Elon Musk will be 54 years old in 2025.'

In [39]:
llm2 = OpenAI(temperature=0, openai_api_key=apikey)

# The tools we'll give the agent access to.  Note that the 'llm-math' tool uses an LLM
tools2 = load_tools(['serpapi', 'llm-math'], llm=llm2, serpapi_api_key=apikey2)
agent2 = initialize_agent(tools2, llm2, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Can't get to work - maybe wrong format?
# Can retrieve search results for US GDP in 2024, but maybe need different method
# to take that result and add 5 to it...
# agent2.run('What was the US GDP in 2024 plus 5?')

In [43]:
# Memory (keep within context window):
# By default - chains are stateless or each query is a new context window
# Can attach "memory" or context to chain.memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run('Mexican')
print(name)



"Casa del Sabor" (House of Flavor)


In [47]:
from pprint import pprint

name = chain.run('Polish')
print(f'name={name.strip()}\n\nchain.memory:')
pprint(chain.memory)
print(f'\n\nchain.memory.buffer:\n{chain.memory.buffer}')

name="Polska Perfection"

chain.memory:
ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Mexican', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Casa del Sabor" (House of Flavor)', additional_kwargs={}, response_metadata={}), HumanMessage(content='Polish', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Polish Palate"', additional_kwargs={}, response_metadata={}), HumanMessage(content='Polish', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Polonez Palace"', additional_kwargs={}, response_metadata={}), HumanMessage(content='Polish', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n"Polska Perfection"', additional_kwargs={}, response_metadata={})]))


chain.memory.buffer:
Human: Mexican
AI: 

"Casa del Sabor" (House of Flavor)
Human: Polish
AI: 

"Polish Palate"
Human: Polish
AI: 

"Polonez Palace"
Human: Polish
AI: 

"Polska Perfection"


In [49]:
from langchain.chains import ConversationChain

conchn = ConversationChain(llm=OpenAI(temperature=0.7, openai_api_key=apikey))
print(conchn.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [50]:
conchn.run('Who won the first super bowl?')

' The first Super Bowl was won by the Green Bay Packers in 1967. They defeated the Kansas City Chiefs with a score of 35-10 at the Los Angeles Memorial Coliseum. The MVP of the game was Bart Starr, the quarterback for the Packers. The game was not yet officially called the Super Bowl at the time, it was known as the AFL-NFL World Championship Game.'

In [51]:
conchn.run('What is 5+5?')

' 5+5 is 10.'

In [52]:
conchn.run('Who was the quarterback for the chiefs?')

' The quarterback for the Kansas City Chiefs during the first Super Bowl was Len Dawson. He completed 16 of 27 passes for 211 yards and one touchdown, but unfortunately, it was not enough to defeat the Green Bay Packers.'

In [57]:
pprint(conchn.memory)

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Who won the first super bowl?', additional_kwargs={}, response_metadata={}), AIMessage(content=' The first Super Bowl was won by the Green Bay Packers in 1967. They defeated the Kansas City Chiefs with a score of 35-10 at the Los Angeles Memorial Coliseum. The MVP of the game was Bart Starr, the quarterback for the Packers. The game was not yet officially called the Super Bowl at the time, it was known as the AFL-NFL World Championship Game.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 5+5?', additional_kwargs={}, response_metadata={}), AIMessage(content=' 5+5 is 10.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Who was the quarterback for the chiefs?', additional_kwargs={}, response_metadata={}), AIMessage(content=' The quarterback for the Kansas City Chiefs during the first Super Bowl was Len Dawson. He completed 16 of 27 passes for 21

In [58]:
print(conchn.memory.buffer)

Human: Who won the first super bowl?
AI:  The first Super Bowl was won by the Green Bay Packers in 1967. They defeated the Kansas City Chiefs with a score of 35-10 at the Los Angeles Memorial Coliseum. The MVP of the game was Bart Starr, the quarterback for the Packers. The game was not yet officially called the Super Bowl at the time, it was known as the AFL-NFL World Championship Game.
Human: What is 5+5?
AI:  5+5 is 10.
Human: Who was the quarterback for the chiefs?
AI:  The quarterback for the Kansas City Chiefs during the first Super Bowl was Len Dawson. He completed 16 of 27 passes for 211 yards and one touchdown, but unfortunately, it was not enough to defeat the Green Bay Packers.


In [59]:
from langchain.memory import ConversationBufferWindowMemory

# Remember 
memory = ConversationBufferWindowMemory(k=1)

conchn = ConversationChain(
    llm=OpenAI(temperature=0.6, openai_api_key=apikey),
    memory=memory,
)

conchn.run('Who won the first super bowl?')

' The first Super Bowl was played in 1967 between the Green Bay Packers and the Kansas City Chiefs. The Green Bay Packers won with a score of 35-10.'

In [60]:
conchn.run('What is 5 + 5?')

' 5 + 5 is 10.'

In [61]:
conchn.run('Who was the quarterback for the chiefs?')

' The quarterback for the Chiefs was Patrick Mahomes during the 2020 NFL season. He led the team to a Super Bowl victory and was named the Super Bowl MVP.'